In [ ]:
import pandas as pd
news = pd.read_csv('../db/preprocessing/news_preprocessed_fixed.csv', nrows=5)
meetings = pd.read_csv('../db/preprocessing/meeting_preprocessed_fixed.csv', nrows=5)
reports = pd.read_csv('../db/preprocessing/final_integrated_full_v2.csv', nrows=5)
press = pd.read_csv('../db/preprocessing/press_preprocessed_fixed.csv', nrows=5)

df_total = pd.concat([news, meetings, reports, press], ignore_index=True)


# 3. 문서 고유 Index
df_total['doc_id'] = df_total.index


final_cols = ['date', 'content', 'tokens', 'category', 'source', 'doc_id']
df_total = df_total[final_cols] 

print(df_total.columns)

In [ ]:
df_total

In [ ]:
!pip install kss

In [ ]:
import kss
from tqdm import tqdm
tqdm.pandas()

df_working = df_total.copy()
df_working['content'] = df_working['content'].progress_apply(kss.split_sentences)
df_sentences = df_working.explode('content').reset_index(drop=True)
df_sentences['tokens'] = None
output_columns = ['doc_id', 'date', 'content', 'tokens', 'category', 'source']
df_sentences = df_sentences[output_columns]


In [ ]:
def ngramize(tokens, max_n=5):
    keep_tags = ['NNG', 'VA', 'VAX', 'MAG', 'VV']
    filtered = [w for w in tokens if w.split('/')[1] in keep_tags]
    all_ngrams = []
    for pos in range(len(filtered)):
        for n in range(1, max_n + 1):
            if pos + n <= len(filtered):
                ngram = ";".join(filtered[pos : pos + n])
                all_ngrams.append({'ngram': ngram, 'start': pos, 'end': pos + n, 'len': n})
                
    final_ngrams = []
    sorted_ngrams = sorted(all_ngrams, key=lambda x: x['len'], reverse=True)
    covered_ranges = set()
    for ngram in sorted_ngrams:
        is_covered = False
        for i in range(ngram['start'], ngram['end']):
            if i in covered_ranges:
                is_covered = True
                break
        if not is_covered:
            final_ngrams.append(ngram['ngram'])
            for i in range(ngram['start'], ngram['end']):
                covered_ranges.add(i)
                
    return final_ngrams

from ekonlpy.sentiment import MPCK
mpck = MPCK()
def get_final_tokens(text):
    basic_tokens = mpck.tokenize(text)

    return ngramize(basic_tokens, max_n=5)

In [ ]:
df_sentences['tokens'] = df_sentences['content'].apply(get_final_tokens)
df_sentences[['content', 'tokens']].head(10)